In [ ]:
# Install required packages
!pip install google-cloud-bigquery google-auth

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

# Set up BigQuery client
from google.cloud import bigquery
import pandas as pd

project_id = 'stone-guard-193511'
client = bigquery.Client(project=project_id)

print("BigQuery client setup complete")

BigQuery client setup complete


In [ ]:
# Check if ML functions are available
def test_bigquery_ml_availability():
    """Test BigQuery ML functions in Colab"""

    # Test 1: Check available models
    try:
        models_query = """
        SELECT
            model_name,
            model_type,
            creation_time
        FROM `stone-guard-193511.INFORMATION_SCHEMA.ML_MODELS`
        LIMIT 5
        """

        models = client.query(models_query).to_dataframe()
        print("✓ Can access ML models metadata")
        print(models)

    except Exception as e:
        print(f"⚠ Models metadata: {e}")

    # Test 2: Try creating a simple model
    try:
        create_model_query = """
        CREATE OR REPLACE MODEL `stone-guard-193511.AbuDhabi_realEstate.simple_model`
        OPTIONS(
            model_type='LINEAR_REG',
            input_label_cols=['Rent']
        ) AS
        SELECT
            Beds,
            Area_in_sqft,
            Rent
        FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
        WHERE Rent IS NOT NULL AND Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
        LIMIT 1000
        """

        job = client.query(create_model_query)
        job.result()
        print("✓ Can create ML models")

    except Exception as e:
        print(f"⚠ Model creation: {e}")

    # Test 3: Try text embedding generation
    try:
        embedding_query = """
        SELECT
            ML.GENERATE_EMBEDDING(
                MODEL `projects/stone-guard-193511/locations/us/models/textembedding-gecko@003`,
                (SELECT 'luxury apartment in marina')
            ).embeddings as test_embedding
        """

        result = client.query(embedding_query).to_dataframe()
        print("✓ Text embeddings work")

    except Exception as e:
        print(f"⚠ Text embeddings: {e}")

test_bigquery_ml_availability()

⚠ Models metadata: 403 Access Denied: Table stone-guard-193511:INFORMATION_SCHEMA.ML_MODELS: User does not have permission to query table stone-guard-193511:INFORMATION_SCHEMA.ML_MODELS, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table stone-guard-193511:INFORMATION_SCHEMA.ML_MODELS: User does not have permission to query table stone-guard-193511:INFORMATION_SCHEMA.ML_MODELS, or perhaps it does not exist.

Location: US
Job ID: 37a6c08d-a584-42b5-91d0-ff7a264d1488

✓ Can create ML models
⚠ Text embeddings: 400 Syntax error: Expected ")" but got identifier `projects/stone-guard-193511/locations/us/models/textembedding-gecko@003` at [4:23]; reason: invalidQuery, location: query, message: Syntax error: Expected ")" but got identifier `projects/stone-guard-193511/locations/us/models/textembedding-gecko@003` at [4:23]

Location: US
Job ID: 9e449695-9365-4e1f-8824-14463b4b077b



In [ ]:
# Test your created model
def test_created_model():
    """Test the ML model you successfully created"""

    # Make predictions using your model
    prediction_query = """
    SELECT
        Address,
        Beds,
        Area_in_sqft,
        predicted_Rent
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.simple_model`,
        (SELECT
            'Test Property' as Address,
            2 as Beds,
            800 as Area_in_sqft
        )
    )
    """

    try:
        predictions = client.query(prediction_query).to_dataframe()
        print("✓ ML predictions work!")
        print(predictions)
        return True
    except Exception as e:
        print(f"Prediction error: {e}")
        return False

model_works = test_created_model()

✓ ML predictions work!
         Address  Beds  Area_in_sqft  predicted_Rent
0  Test Property     2           800    33806.236728


In [ ]:
# Create a property category classifier
def create_property_classifier():
    """Create a model to classify property types"""

    classifier_query = """
    CREATE OR REPLACE MODEL `stone-guard-193511.AbuDhabi_realEstate.property_classifier`
    OPTIONS(
        model_type='LOGISTIC_REG',
        input_label_cols=['Rent_category']
    ) AS
    SELECT
        Beds,
        Area_in_sqft,
        Rent,
        Rent_category
    FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
    WHERE Rent_category IS NOT NULL
        AND Beds IS NOT NULL
        AND Area_in_sqft IS NOT NULL
    """

    try:
        job = client.query(classifier_query)
        job.result()
        print("✓ Property classifier created!")
        return True
    except Exception as e:
        print(f"Classifier creation failed: {e}")
        return False

classifier_created = create_property_classifier()

✓ Property classifier created!


In [ ]:
def create_complete_bigquery_ml_system():
    """Build full system using available BigQuery ML functions"""

    # 1. Property value prediction
    value_analysis_query = """
    SELECT
        Address,
        Location,
        Rent,
        Beds,
        Area_in_sqft,
        predicted_Rent,
        ABS(Rent - predicted_Rent) as price_difference,
        CASE
            WHEN Rent < predicted_Rent * 0.9 THEN 'Undervalued'
            WHEN Rent > predicted_Rent * 1.1 THEN 'Overvalued'
            ELSE 'Market Rate'
        END as value_assessment
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.simple_model`,
        (SELECT Address, Location, Rent, Beds, Area_in_sqft
         FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
         WHERE Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
         LIMIT 50)
    )
    ORDER BY price_difference DESC
    """

    # 2. Property classification
    classification_query = """
    SELECT
        Address,
        Location,
        Rent,
        Beds,
        Area_in_sqft,
        predicted_Rent_category,
        predicted_Rent_category_probs
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.property_classifier`,
        (SELECT Address, Location, Rent, Beds, Area_in_sqft
         FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
         WHERE Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
         LIMIT 20)
    )
    """

    try:
        # Execute value analysis
        value_results = client.query(value_analysis_query).to_dataframe()
        print("Value Analysis Results:")
        print(value_results[['Address', 'Rent', 'predicted_Rent', 'value_assessment']].head())

        # Execute classification
        class_results = client.query(classification_query).to_dataframe()
        print("\nProperty Classification Results:")
        print(class_results[['Address', 'Rent', 'predicted_Rent_category']].head())

        return value_results, class_results

    except Exception as e:
        print(f"ML analysis failed: {e}")
        return None, None

# Run complete ML analysis
if model_works and classifier_created:
    value_analysis, classification_results = create_complete_bigquery_ml_system()

Value Analysis Results:
                                             Address    Rent  predicted_Rent  \
0  1 Residences Building 1, 1 Residences, Wasl 1,...  110000    34176.532068   
1  15 Northside Tower 1, 15 Northside, Business B...   92000    32018.604050   
2  15 Northside Tower 1, 15 Northside, Business B...   90000    32018.604050   
3  29 Boulevard 2, 29 Boulevard, Downtown Dubai, ...   90000    32495.306097   
4  8 Boulevard Walk, Mohammad Bin Rashid Boulevar...   90000    32712.375780   

  value_assessment  
0       Overvalued  
1       Overvalued  
2       Overvalued  
3       Overvalued  
4       Overvalued  

Property Classification Results:
                                             Address   Rent  \
0         18 Villas Complex, Khalifa City, Abu Dhabi  25000   
1  29 Boulevard Podium, 29 Boulevard, Downtown Du...  80000   
2         18 Villas Complex, Khalifa City, Abu Dhabi  30000   
3  29 Boulevard 1, 29 Boulevard, Downtown Dubai, ...  75000   
4  15 Northside Towe

---------------------------------------------------------------------------

In [ ]:
YOUR_ACCESS_KEY='ycawAMyaHjW3EdWYl-KWuH0zjUnfECraOr4HKbjGA0k'

In [ ]:
import requests

# Set up the API request
url = "https://api.unsplash.com/search/photos"
params = {
    "query": "business bay apartment dubai",
    "client_id": "ycawAMyaHjW3EdWYl-KWuH0zjUnfECraOr4HKbjGA0k",  # Your access key
    "per_page": 15  # You can adjust this
}

# Make the request
response = requests.get(url, params=params)

# Parse the results
data = response.json()

# Print image URLs
for i, result in enumerate(data["results"], 1):
    print(f"{i}. {result['urls']['regular']}")

1. https://images.unsplash.com/photo-1719474818087-f334f1e92985?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwxfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
2. https://images.unsplash.com/photo-1719474818095-2d75b8b57251?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwyfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
3. https://images.unsplash.com/photo-1719474815671-08411b0ea840?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwzfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
4. https://images.unsplash.com/photo-1719474814907-7fb947f865aa?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHw0fHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
5. https://images.unsplash.com/photo

In [ ]:
import requests

def fetch_unsplash_images(query="business bay apartment dubai", per_page=15):
    """Fetch Unsplash image URLs for a location/type"""
    access_key = "ycawAMyaHjW3EdWYl-KWuH0zjUnfECraOr4HKbjGA0k"
    url = "https://api.unsplash.com/search/photos"

    params = {
        "query": query,
        "client_id": access_key,
        "per_page": per_page
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return [img["urls"]["regular"] for img in data["results"]]
    else:
        print("Error:", response.text)
        return []

# ✅ Save the output into this variable
unsplash_images = fetch_unsplash_images()

# Optional: Preview the results
for i, img_url in enumerate(unsplash_images, 1):
    print(f"{i}. {img_url}")


1. https://images.unsplash.com/photo-1719474818087-f334f1e92985?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwxfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
2. https://images.unsplash.com/photo-1719474818095-2d75b8b57251?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwyfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
3. https://images.unsplash.com/photo-1719474815671-08411b0ea840?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHwzfHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
4. https://images.unsplash.com/photo-1719474814907-7fb947f865aa?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3OTY2MDJ8MHwxfHNlYXJjaHw0fHxidXNpbmVzcyUyMGJheSUyMGFwYXJ0bWVudCUyMGR1YmFpfGVufDB8fHx8MTc1Nzg4MzY5OXww&ixlib=rb-4.1.0&q=80&w=1080
5. https://images.unsplash.com/photo

-------------------------------------------------------------------------------

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="stone-guard-193511")

def create_complete_bigquery_ml_system():
    """Run ML predictions for rent value and classification"""

    value_query = """
    SELECT
        Address,
        Location,
        Rent,
        Beds,
        Area_in_sqft,
        predicted_Rent,
        ABS(Rent - predicted_Rent) AS price_difference,
        CASE
            WHEN Rent < predicted_Rent * 0.9 THEN 'Undervalued'
            WHEN Rent > predicted_Rent * 1.1 THEN 'Overvalued'
            ELSE 'Market Rate'
        END AS ML_Assessment
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.simple_model`,
        (SELECT Address, Location, Rent, Beds, Area_in_sqft
         FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
         WHERE Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
         LIMIT 50)
    )
    """

    class_query = """
    SELECT
        Address,
        Location,
        Rent,
        Beds,
        Area_in_sqft,
        predicted_Rent_category
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.property_classifier`,
        (SELECT Address, Location, Rent, Beds, Area_in_sqft
         FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
         WHERE Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
         LIMIT 50)
    )
    """

    value_results = client.query(value_query).to_dataframe()
    class_results = client.query(class_query).to_dataframe()

    return value_results, class_results

# Run ML systems
value_results, class_results = create_complete_bigquery_ml_system()


In [ ]:
import pandas as pd
import random

def build_multimodal_dataset(value_df, class_df, image_urls):
    """Join value & classification outputs with Unsplash imagery"""

    df = pd.merge(value_df, class_df[['Address', 'predicted_Rent_category']], on='Address', how='left')

    # Add unsplash image per row
    df['Interior_Image_URL'] = [random.choice(image_urls) for _ in range(len(df))]

    return df

# Build final multimodal dataframe
multimodal_df = build_multimodal_dataset(value_results, class_results, unsplash_images)


In [ ]:
multimodal_df.head()

,Address,Location,Rent,Beds,Area_in_sqft,predicted_Rent,price_difference,ML_Assessment,predicted_Rent_category,Interior_Image_URL
0,"1 Residences Building 1, 1 Residences, Wasl 1,...",Bur Dubai,110000,0,887,34176.532068,75823.467932,Overvalued,Low,https://images.unsplash.com/photo-171947481490...
1,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481538...
2,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-167031295791...
3,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-1559601140-c...
4,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481490...


In [ ]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-pm2afr8b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-pm2afr8b
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=94322d91ecef6e05c59519ca4ef627772ec1deff75c8ae7c1bdee7f83d76e996
  Stored in directory: /tmp/pip-ephem-wheel-cache-68roagf6/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
from PIL import Image
import requests
from io import BytesIO
import torch
import clip  # Requires openai/CLIP
import pandas as pd

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def generate_embedding_from_url(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_input)
        return image_features[0].cpu().numpy()
    except Exception as e:
        print(f"Failed for {image_url}: {e}")
        return None


100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 63.9MiB/s]


In [ ]:
multimodal_df["image_embedding"] = multimodal_df["Interior_Image_URL"].apply(generate_embedding_from_url)


In [ ]:
multimodal_df.head()

,Address,Location,Rent,Beds,Area_in_sqft,predicted_Rent,price_difference,ML_Assessment,predicted_Rent_category,Interior_Image_URL,image_embedding
0,"1 Residences Building 1, 1 Residences, Wasl 1,...",Bur Dubai,110000,0,887,34176.532068,75823.467932,Overvalued,Low,https://images.unsplash.com/photo-171947481490...,"[-0.11855139, -0.022390675, -0.036633193, 0.07..."
1,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481538...,"[-0.08038054, 0.21260427, 0.28613088, -0.22104..."
2,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-167031295791...,"[-0.048842635, 0.032427218, 0.6086046, -0.5715..."
3,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-1559601140-c...,"[-0.07999983, 0.2108963, 0.3953592, -0.1635951..."
4,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481490...,"[-0.11855139, -0.022390675, -0.036633193, 0.07..."


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/Colab Notebooks'

In [ ]:
# Save to the specified path
path = '/content/drive/MyDrive/Colab Notebooks/multimodal_df.csv'
multimodal_df.to_csv(path, index=False)

print(f"Saved to {path}")


Saved to /content/drive/MyDrive/Colab Notebooks/multimodal_df.csv


In [ ]:
import pandas as pd

In [ ]:
multimodal_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/multimodal_df.csv')

In [ ]:
multimodal_df.head()

,Address,Location,Rent,Beds,Area_in_sqft,predicted_Rent,price_difference,ML_Assessment,predicted_Rent_category,Interior_Image_URL,image_embedding
0,"1 Residences Building 1, 1 Residences, Wasl 1,...",Bur Dubai,110000,0,887,34176.532068,75823.467932,Overvalued,Low,https://images.unsplash.com/photo-171947481567...,[-1.93533421e-01 1.39956325e-01 1.07791582e-...
1,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481490...,[-1.18551388e-01 -2.23906748e-02 -3.66331935e-...
2,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481547...,[-3.28644723e-01 4.27298754e-01 1.13669023e-...
3,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-173532086423...,[ 8.88767838e-02 1.13252535e-01 3.12444746e-...
4,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481808...,[-9.05162618e-02 -2.21680254e-02 4.13374186e-...


In [ ]:
print(multimodal_df.columns.tolist())


['Address', 'Location', 'Rent', 'Beds', 'Area_in_sqft', 'predicted_Rent', 'price_difference', 'ML_Assessment', 'predicted_Rent_category', 'Interior_Image_URL', 'image_embedding']


In [ ]:
def semantic_search(query, df, top_k=5):
    query = query.lower()
    results = []

    for idx, row in df.iterrows():
        score = 0
        text = f"{row['Address']} {row['Location']} {row['ML_Assessment']}".lower()

        # Match predicted rent category semantically
        category = str(row.get('predicted_Rent_category', '')).lower()
        if any(word in query for word in ['luxury', 'high-end']):
            score += 2 if category == 'high' else 0
        elif 'affordable' in query or 'cheap' in query:
            score += 2 if category == 'low' else 0
        elif 'standard' in query or 'moderate' in query:
            score += 2 if category == 'medium' else 0

        # Bedroom match
        if 'bedroom' in query or 'studio' in query:
            try:
                if 'studio' in query and row['Beds'] == 0:
                    score += 2
                else:
                    for word in query.split():
                        if word.isdigit() and int(word) == row['Beds']:
                            score += 2
            except:
                pass

        # General token match
        if any(w in text for w in query.split()):
            score += 1

        if score > 0:
            results.append({**row, 'relevance_score': score})

    return pd.DataFrame(results).sort_values('relevance_score', ascending=False).head(top_k)



In [ ]:
semantic_search("affordable 3 bedroom in Dubai", multimodal_df)


,Address,Location,Rent,Beds,Area_in_sqft,predicted_Rent,price_difference,ML_Assessment,predicted_Rent_category,Interior_Image_URL,image_embedding,relevance_score
0,"1 Residences Building 1, 1 Residences, Wasl 1,...",Bur Dubai,110000,0,887,34176.532068,75823.467932,Overvalued,Low,https://images.unsplash.com/photo-171947481490...,"[-0.11855139, -0.022390675, -0.036633193, 0.07...",3
1,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481538...,"[-0.08038054, 0.21260427, 0.28613088, -0.22104...",3
2,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-167031295791...,"[-0.048842635, 0.032427218, 0.6086046, -0.5715...",3
3,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-1559601140-c...,"[-0.07999983, 0.2108963, 0.3953592, -0.1635951...",3
4,"15 Northside Tower 1, 15 Northside, Business B...",Business Bay,85000,0,371,31980.297635,53019.702365,Overvalued,Low,https://images.unsplash.com/photo-171947481490...,"[-0.11855139, -0.022390675, -0.036633193, 0.07...",3


---------------------------------------------------------------------------

In [ ]:
import numpy as np

In [ ]:
bucket_images = [
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay1.jpg", "name": "dubai_pay1.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay12.jpg", "name": "dubai_pay12.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay13.jpg", "name": "dubai_pay13.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay15.jpg", "name": "dubai_pay15.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay2.jpg", "name": "dubai_pay2.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay3.jpg", "name": "dubai_pay3.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay5.jpg", "name": "dubai_pay5.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay4.jpg", "name": "dubai_pay4.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay6.jpg", "name": "dubai_pay6.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay7.jpg", "name": "dubai_pay7.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay8.jpg", "name": "dubai_pay8.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay9.jpg", "name": "dubai_pay9.jpg"},
    {"uri": "https://storage.googleapis.com/real_estate_imagess/dubai_pay10.jpg", "name": "dubai_pay10.jpg"}
]



In [ ]:
def create_complete_multimodal_system_colab():
    """Build multimodal system combining BigQuery ML + Unsplash images"""

    # 1. Get properties from BigQuery
    properties_query = """
    SELECT
        Address, Location, Rent, Beds, Baths, Area_in_sqft,
        Rent_category, Latitude, Longitude
    FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
    WHERE Rent IS NOT NULL AND Beds IS NOT NULL
    LIMIT 100
    """

    properties_df = client.query(properties_query).to_dataframe()

    # 2. Add ML predictions
    ml_predictions_query = """
    SELECT
        Address, Location, Rent, Beds, Area_in_sqft,
        predicted_Rent,
        CASE
            WHEN Rent < predicted_Rent * 0.9 THEN 'Undervalued'
            WHEN Rent > predicted_Rent * 1.1 THEN 'Overvalued'
            ELSE 'Market Rate'
        END as ml_assessment
    FROM ML.PREDICT(
        MODEL `stone-guard-193511.AbuDhabi_realEstate.simple_model`,
        (SELECT Address, Location, Rent, Beds, Area_in_sqft
         FROM `stone-guard-193511.AbuDhabi_realEstate.abu_dhabi_properties`
         WHERE Beds IS NOT NULL AND Area_in_sqft IS NOT NULL
         LIMIT 50)
    )
    """

    ml_results = client.query(ml_predictions_query).to_dataframe()

    # 3. Create multimodal property database
    multimodal_properties = []

    for idx, prop in ml_results.iterrows():
        # Assign interior image based on property characteristics
        if prop['Rent'] > 200000:
            # High-end properties get premium images
            image_choice = bucket_images[0:4]  # First 4 images for luxury
        elif prop['Rent'] > 100000:
            # Mid-range properties
            image_choice = bucket_images[4:8]  # Middle images
        else:
            # Budget properties
            image_choice = bucket_images[8:]   # Last images

        selected_image = np.random.choice(image_choice)

        multimodal_properties.append({
            'address': prop['Address'],
            'location': prop['Location'],
            'rent': prop['Rent'],
            'beds': prop['Beds'],
            'predicted_rent': prop['predicted_Rent'],
            'ml_assessment': prop['ml_assessment'],
            'interior_image_url': selected_image['uri'],
            'interior_image_name': selected_image['name'],
            'data_sources': ['BigQuery_ML', 'Structured_Data', 'Unsplash_Interior']
        })

    return pd.DataFrame(multimodal_properties)

# Create the system
multimodal_df = create_complete_multimodal_system_colab()

# 4. Natural language search with ML + images
def ml_powered_search(query, multimodal_data):
    """Search with ML insights and image results"""

    query_lower = query.lower()
    results = []

    for idx, prop in multimodal_data.iterrows():
        score = 0

        # ML-based scoring
        if 'undervalued' in query_lower and prop['ml_assessment'] == 'Undervalued':
            score += 5
        elif 'investment' in query_lower and prop['ml_assessment'] == 'Undervalued':
            score += 4

        # Traditional search
        if 'luxury' in query_lower and prop['rent'] > 150000:
            score += 3
        elif 'affordable' in query_lower and prop['rent'] < 80000:
            score += 3

        if score > 0:
            results.append({
                'address': prop['address'],
                'rent': prop['rent'],
                'predicted_rent': prop['predicted_rent'],
                'ml_assessment': prop['ml_assessment'],
                'interior_image': prop['interior_image_url'],
                'score': score
            })

    return sorted(results, key=lambda x: x['score'], reverse=True)[:5]

# Test the complete system
test_results = ml_powered_search("undervalued luxury property with interior photos", multimodal_df)

for i, result in enumerate(test_results, 1):
    print(f"\n{i}. {result['address']}")
    print(f"   Rent: AED {result['rent']:,}")
    print(f"   ML Assessment: {result['ml_assessment']}")
    print(f"   Interior Image: {result['interior_image']}")


1. 18 Villas Complex, Khalifa City, Abu Dhabi
   Rent: AED 30,000
   ML Assessment: Undervalued
   Interior Image: https://storage.googleapis.com/real_estate_imagess/dubai_pay10.jpg

2. 18 Villas Complex, Khalifa City, Abu Dhabi
   Rent: AED 25,000
   ML Assessment: Undervalued
   Interior Image: https://storage.googleapis.com/real_estate_imagess/dubai_pay7.jpg

3. 5102 Muwaileh Building, Muwailih Commercial, Sharjah
   Rent: AED 15,000
   ML Assessment: Undervalued
   Interior Image: https://storage.googleapis.com/real_estate_imagess/dubai_pay8.jpg

4. 5102 Muwaileh Building, Muwailih Commercial, Sharjah
   Rent: AED 23,000
   ML Assessment: Undervalued
   Interior Image: https://storage.googleapis.com/real_estate_imagess/dubai_pay7.jpg

5. 5102 Muwaileh Building, Muwailih Commercial, Sharjah
   Rent: AED 23,000
   ML Assessment: Undervalued
   Interior Image: https://storage.googleapis.com/real_estate_imagess/dubai_pay8.jpg


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="stone-guard-193511")  # <-- your actual project


In [ ]:
def semantic_search(query, df, top_k=5):
    query = query.lower()
    results = []

    for idx, row in df.iterrows():
        score = 0
        text = f"{row['Address']} {row['Location']} {row['ML_Assessment']} {row['predicted_Rent_category']}".lower()

        if any(word in query for word in ['luxury', 'high-end']):
            score += 2 if row['predicted_Rent_category'] == 'High' else 0
        elif 'affordable' in query or 'cheap' in query:
            score += 2 if row['predicted_Rent_category'] == 'Low' else 0
        if str(row['Beds']) in query:
            score += 1
        if any(w in text for w in query.split()):
            score += 1

        if score > 0:
            results.append({**row, 'relevance_score': score})

    return pd.DataFrame(results).sort_values('relevance_score', ascending=False).head(top_k)


In [ ]:
semantic_search("affordable 2 bedroom in Dubai", multimodal_df)


KeyError: 'Address'